In [2]:
import requests
import urllib.parse
import requests
import os
import bs4
import pandas as pd
from censusgeocode import CensusGeocode
import numpy as np

In [398]:
filename1 = 'Data/311_Service_Requests_-_Vacant_and_Abandoned_Buildings_Reported'
filename4 = 'Data/311_Service_Requests_-_Sanitation_Code_Complaints'

df1 = pd.read_csv(filename1 + '.csv', skiprows = [1], encoding = "ISO-8859-1")
df1 = df1.rename(columns = {'LATITUDE':'Latitude', 'LONGITUDE':'Longitude'})
df4 = pd.read_csv(filename4 + '.csv', skiprows = [1], encoding = "ISO-8859-1")

In [399]:
def fips_df(df):
    '''
    This function returns the separate elements of FIPS: state, county, tract and block group
    
    Input:
        a dataframe with latitutde and logitude columns
    
    Returns:
        a tuple with elements of FIPS
    '''
    
    df = df[:100] # Using analysis on first 100 rows from 2016 data

    cg = CensusGeocode()

    latitude = df["Latitude"]
    longitude = df["Longitude"]

    latitude = list(latitude)
    longitude = list(longitude)

    #Deal with nans
    long_list1 = []
    for i in range(len(longitude)):
        longitude1 = str(longitude[i])
        if longitude1 != "nan":
            longitude1 = float(longitude1)
        long_list1.append(longitude1)
    longitude = long_list1

    lat_list1 = []

    for i in range(len(latitude)):
        latitude1 = str(latitude[i])
        if latitude1 != "nan":
            latitude1 = float(latitude1)
        lat_list1.append(latitude1)
    latitude = lat_list1

    #GET FIPS
    #fips_list = []
    attributes_list = []

    for i in range(len(latitude)):
        if latitude[i] != "nan":
            result = cg.coordinates(x=longitude[i], y=latitude[i])
            #fips = result[0]['2010 Census Blocks'][0]["GEOID"]
            state = result[0]['2010 Census Blocks'][0]["STATE"]
            county = result[0]['2010 Census Blocks'][0]["COUNTY"]
            tract = result[0]['2010 Census Blocks'][0]["TRACT"]
            block = result[0]['2010 Census Blocks'][0]["BLKGRP"]
            #fips_list.append(fips)
            tuple1 = (state,county,tract,block)
            attributes_list.append(tuple1)
            print(tuple1)

        else:
            fips = "nan"
            state = "nan"
            county = "nan"
            tract = "nan"
            block = "nan"
            #fips_list.append(fips)
            tuple1 = (state,county,tract,block)
            attributes_list.append(tuple1)
            print(tuple1)
    return attributes_list

In [400]:
def get_request(url):
    '''
    Open a connection to the specified URL and if successful
    read the data.

    Inputs:
        url: must be an absolute URL
    
    Outputs: 
        request object or None
    '''
    
    try:
        r = requests.get(url)
        if r.status_code == 404 or r.status_code == 403:
            r = None
    except:
        # fail on any kind of error
        r = None

    return r

In [402]:
#var1 = Population of two or more races
#var2 = Household Income by Gross Rent As A Percentage of Household Income in the Past 12 Months
#var3 = Owner occupied houses

def variables_lists(df):
    '''
    This function returns the values of three different variables aforementioned and the Block group.
    
    Input:
        a dataframe
        
    Returns:
        a tuple containing three demographic-related variables and the Block group of each complaint.
    '''
    
    var1_list = []
    var2_list = []
    var3_list = []
    block_list = []
    attributes_list = fips_df(df)
    for a,b,c,d in attributes_list:
        var1 = get_request('http://api.census.gov/data/2015/acs5?get=NAME,C02003_012E&for=block+group:' + d + '&in=state:' + a + '&in=county:' + b + '&in=tract:' + c + '&&key=0e11756a48f5a8ec01c921ac7afbbf7bed1e84e6')
        var2 = get_request('http://api.census.gov/data/2015/acs5?get=NAME,B25074_001E&for=block+group:' + d + '&in=state:' + a + '&in=county:' + b + '&in=tract:' + c + '&&key=0e11756a48f5a8ec01c921ac7afbbf7bed1e84e6')
        var3 = get_request('http://api.census.gov/data/2015/acs5?get=NAME,B25009_002E&for=block+group:' + d + '&in=state:' + a + '&in=county:' + b + '&in=tract:' + c + '&&key=0e11756a48f5a8ec01c921ac7afbbf7bed1e84e6')
        block_list.append(int(d))
        for i in var1.json():
            if i[1] != 'C02003_012E':
                var1_list.append(int(i[1]))
        for j in var2.json():
            if j[1] != 'B25074_001E':
                var2_list.append(int(j[1]))
        for k in var3.json():
            if k[1] != 'B25009_002E':
                var3_list.append(int(k[1]))
                
    return var1_list, var2_list, var3_list, block_list

In [403]:
def update_vars_df(df):
    '''
    Appends values of three demographic-related variables into the original datafram
    
    Input:
        a dataframe
    
    Returns:
        updated dataframe
    '''
    
    var1_list, var2_list, var3_list, block_list = variables_lists(df)
    
    df['Block'] = pd.DataFrame(np.array(block_list))
    df['Population of Multiple Races'] = pd.DataFrame(np.array(var1_list))
    df['Household Income by Gross Rent ($000)'] = pd.DataFrame(np.array(var2_list))
    df['Owner Occupied Houses'] = pd.DataFrame(np.array(var3_list))
    return df

In [404]:
df1 = update_vars_df(df1) # Calculating FIPS for vacant building data

('17', '031', '660400', '3')
('17', '031', '670200', '1')
('17', '031', '710900', '2')
('17', '031', '710800', '6')
('17', '031', '420600', '1')
('17', '031', '530503', '2')
('17', '031', '250500', '2')
('17', '031', '530200', '5')
('17', '031', '690400', '1')
('17', '031', '700501', '4')
('17', '031', '460100', '2')
('17', '031', '440700', '1')
('17', '031', '190300', '2')
('17', '031', '670300', '1')
('17', '031', '671500', '2')
('17', '031', '390300', '1')
('17', '031', '661000', '1')
('17', '031', '672000', '3')
('17', '031', '680500', '2')
('17', '031', '660800', '3')
('17', '031', '660600', '5')
('17', '031', '843400', '1')
('17', '031', '252102', '2')
('17', '031', '490901', '3')
('17', '031', '680500', '2')
('17', '031', '680500', '2')
('17', '031', '260700', '2')
('17', '031', '191100', '5')
('17', '031', '460301', '1')
('17', '031', '460301', '1')
('17', '031', '540102', '3')
('17', '031', '260700', '1')
('17', '031', '260400', '1')
('17', '031', '681400', '2')
('17', '031', 

In [405]:
df1[:100]

,SERVICE REQUEST TYPE,SERVICE REQUEST NUMBER,DATE SERVICE REQUEST WAS RECEIVED,"LOCATION OF BUILDING ON THE LOT (IF GARAGE, CHANGE TYPE CODE TO BGD).",IS THE BUILDING DANGEROUS OR HAZARDOUS?,IS BUILDING OPEN OR BOARDED?,"IF THE BUILDING IS OPEN, WHERE IS THE ENTRY POINT?",IS THE BUILDING CURRENTLY VACANT OR OCCUPIED?,IS THE BUILDING VACANT DUE TO FIRE?,"ANY PEOPLE USING PROPERTY? (HOMELESS, CHILDEN, GANGS)",...,Ward,Police District,Community Area,Latitude,Longitude,Location,Block,Population of Multiple Races,Household Income by Gross Rent ($000),Owner Occupied Houses
0,Vacant/Abandoned Building,16-00027497,01/03/2016,Front,NaN,Open,NaN,Vacant,False,False,...,23.0,8.0,66.0,41.778832,-87.712726,"(41.778832091928, -87.7127256067861)",3.0,0.0,71.0,171.0
1,Vacant/Abandoned Building,16-00035418,01/03/2016,Front,NaN,Open,"CALLER DOESN'T KNOW THE EXACT ADDRESS, CALLER ...",Vacant,False,False,...,16.0,7.0,67.0,41.786764,-87.663167,"(41.786764299374006, -87.66316688619024)",1.0,0.0,153.0,124.0
2,Vacant/Abandoned Building,16-00031311,01/03/2016,Front,NaN,Open,FRONT ENTRY,Vacant,False,False,...,21.0,6.0,71.0,41.742258,-87.641018,"(41.74225776568885, -87.64101798247111)",2.0,13.0,279.0,234.0
3,Vacant/Abandoned Building,16-00035051,01/03/2016,Rear,NaN,Open,REAR,Vacant,False,False,...,21.0,6.0,71.0,41.746654,-87.652421,"(41.74665406968148, -87.65242142853317)",6.0,0.0,227.0,100.0
4,Vacant/Abandoned Building,16-00056596,01/04/2016,Front,NaN,Open,NaN,Vacant,False,False,...,20.0,3.0,42.0,41.783886,-87.615632,"(41.78388564277491, -87.61563244911125)",1.0,15.0,371.0,33.0
5,Vacant/Abandoned Building,16-00054380,01/04/2016,Front,NaN,Open,NaN,Vacant,False,True,...,34.0,5.0,53.0,41.668086,-87.640383,"(41.66808588888048, -87.64038251498134)",2.0,10.0,166.0,376.0
6,Vacant/Abandoned Building,16-00062202,01/04/2016,Front,NaN,Open,FRONT DOOR OPEN,Vacant,False,False,...,29.0,25.0,25.0,41.909606,-87.786648,"(41.9096055811526, -87.78664751486437)",2.0,67.0,81.0,342.0
7,Vacant/Abandoned Building,16-00052007,01/04/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.0,5.0,53.0,41.680829,-87.628750,"(41.6808288273904, -87.62875014237845)",5.0,23.0,236.0,103.0
8,Vacant/Abandoned Building,16-00055970,01/04/2016,Front,NaN,Open,NaN,Vacant,False,True,...,20.0,3.0,69.0,41.770520,-87.616998,"(41.770519924959366, -87.61699833276981)",1.0,0.0,226.0,58.0
9,Vacant/Abandoned Building,16-00066354,01/04/2016,Front,NaN,Open,NaN,Vacant,False,True,...,18.0,8.0,70.0,41.745824,-87.697401,"(41.745824149949684, -87.69740124886198)",4.0,0.0,277.0,400.0


In [488]:
df1_small_1 = df1[['Population of Multiple Races', 'Household Income by Gross Rent ($000)', 'Owner Occupied Houses']].groupby(df1['Block']).mean().reset_index()

In [489]:
df1_small_2 = df1['Location'].groupby(df1['Block']).count().reset_index()

In [494]:
df1_result = pd.merge(df1_small_1, df1_small_2, on='Block', how="outer")

In [594]:
# group by based on Block Group
df1_result = df1_result.rename(columns = {'Location':'Count of vacant requests'})
df1_result

,Block,Population of Multiple Races,Household Income by Gross Rent ($000),Owner Occupied Houses,Count of vacant requests
0,1.0,15.176471,245.441176,136.794118,34
1,2.0,15.542857,242.371429,175.028571,35
2,3.0,18.470588,217.294118,192.176471,17
3,4.0,5.888889,353.888889,162.333333,9
4,5.0,22.333333,293.000000,152.333333,3
5,6.0,0.000000,227.000000,100.000000,1
6,7.0,15.000000,304.000000,282.000000,1


The blocks with the highest instances of vacant/abandoned buildings are Block 1 and Block 2.
These blocks have a high rate of mixed race population, but low household income and relatively lower proportion
of occupied houses (i.e. more houses are on rent). One interpretation of this is that most of the immigrants
live in Block 1 and Block 2, and do not have well to paid jobs. There's also a high mobility specially in Block 1 as most of the people tend to live on rent.

The blocks with the lower instances of vacant/abandoned buildings are Block 4, Block 5 and Block 6. In these blocks,
the average household income is high. However, the population of mixed races and proportion of occupied houses is still the same, except in Block 6. Block 6 is characterized by zero mix race of population (missing data?) and a very low owner occupied houses. Block 6 may comprise of millenials and young people who tend to live for long periods of time and therefore generate less complaints about vacant/abandoned buildings.

In [591]:
df1.groupby(['DATE SERVICE REQUEST WAS RECEIVED'])['Block'].max().reset_index()[:14]

,DATE SERVICE REQUEST WAS RECEIVED,Block
0,2016-01-03,6.0
1,2016-01-04,5.0
2,2016-01-05,5.0
3,2016-01-06,3.0
4,2016-01-07,4.0
5,2016-01-08,4.0
6,2016-01-09,2.0
7,2016-01-10,1.0
8,2016-01-11,3.0
9,2016-01-12,3.0


I have run this analysis on a small sample of dates (first two weeks of Jan). It shows that overtime, the maximum number of requests on daily basis is received by a different Block. The occurences of Block 2 and 3 are more frequent.

In [407]:
df4 = update_vars_df(df4) # calculating FIPS for sanitation data

('17', '031', '440300', '5')
('17', '031', '839100', '2')
('17', '031', '020601', '3')
('17', '031', '051300', '1')
('17', '031', '630400', '4')
('17', '031', '081403', '1')
('17', '031', '530501', '1')
('17', '031', '700302', '2')
('17', '031', '230700', '2')
('17', '031', '430101', '1')
('17', '031', '440202', '1')
('17', '031', '611200', '1')
('17', '031', '241100', '2')
('17', '031', '510200', '1')
('17', '031', '650400', '3')
('17', '031', '240800', '2')
('17', '031', '222200', '2')
('17', '031', '251600', '2')
('17', '031', '150200', '4')
('17', '031', '150200', '4')
('17', '031', '831100', '4')
('17', '031', '711200', '4')
('17', '031', '140602', '1')
('17', '031', '242600', '1')
('17', '031', '700301', '2')
('17', '031', '750300', '2')
('17', '031', '740100', '4')
('17', '031', '830500', '2')
('17', '031', '834400', '3')
('17', '031', '837000', '1')
('17', '031', '310800', '2')
('17', '031', '310800', '2')
('17', '031', '310800', '2')
('17', '031', '310800', '2')
('17', '031', 

In [408]:
df4[:100]

,Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,What is the Nature of this Code Violation?,Street Address,ZIP Code,X Coordinate,Y Coordinate,Ward,Police District,Community Area,Latitude,Longitude,Location,Block,Population of Multiple Races,Household Income by Gross Rent ($000),Owner Occupied Houses
0,01/01/2016,Completed,01/04/2016,16-00008105,Sanitation Code Violation,Garbage in yard,7938 S WABASH AVE,60619.0,1.178087e+06,1.852392e+06,6,6,44,41.750278,-87.622979,"(41.750278002798325, -87.62297857190532)",5.0,0.0,157.0,222.0
1,01/01/2016,Completed,01/04/2016,16-00008183,Sanitation Code Violation,Construction Site Cleanliness/Fence,442 W WACKER DR ER,60607.0,1.174083e+06,1.898170e+06,25,1,32,41.875985,-87.636291,"(41.87598480231486, -87.6362905484112)",2.0,49.0,471.0,543.0
2,01/01/2016,Completed,01/04/2016,16-00009911,Sanitation Code Violation,Dumpster not being emptied,2712 W ARTHUR AVE,60645.0,1.157018e+06,1.943020e+06,50,24,2,41.999420,-87.697727,"(41.999419638611165, -87.69772708238521)",3.0,0.0,180.0,162.0
3,01/01/2016,Completed,01/04/2016,16-00011795,Sanitation Code Violation,Dumpster not being emptied,3030 N HONORE ST,60657.0,1.163584e+06,1.920281e+06,32,19,5,41.936888,-87.674216,"(41.93688765836184, -87.67421593086208)",1.0,28.0,165.0,303.0
4,01/01/2016,Completed,01/05/2016,16-00005916,Sanitation Code Violation,Other,5258 S WHIPPLE ST,60632.0,1.156936e+06,1.869613e+06,14,9,63,41.797987,-87.700022,"(41.797987190613256, -87.70002151020466)",4.0,7.0,338.0,168.0
5,01/01/2016,Completed,01/05/2016,16-00008826,Sanitation Code Violation,Other,377 N NB OUTER LAKE SHORE DR,60601.0,1.180120e+06,1.902848e+06,42,18,8,41.888685,-87.613981,"(41.88868520186334, -87.61398112884407)",1.0,82.0,1274.0,985.0
6,01/01/2016,Completed,01/06/2016,16-00005194,Sanitation Code Violation,Overflowing carts,12000 S PRINCETON AVE,60628.0,1.176442e+06,1.825356e+06,34,5,53,41.676124,-87.629816,"(41.67612352544859, -87.62981637199104)",1.0,0.0,282.0,160.0
7,01/01/2016,Completed,01/11/2016,16-00007120,Sanitation Code Violation,Dog feces in yard,8612 S KOSTNER AVE,60652.0,1.148594e+06,1.847073e+06,18,8,70,41.736297,-87.731193,"(41.73629660780156, -87.73119328676863)",2.0,15.0,21.0,335.0
8,01/01/2016,Completed,01/13/2016,16-00008044,Sanitation Code Violation,Dog feces in yard,3678 W GRAND AVE,60651.0,1.151463e+06,1.907935e+06,26,25,23,41.903255,-87.719086,"(41.903255176586924, -87.71908648670426)",2.0,18.0,104.0,152.0
9,01/02/2016,Completed,01/04/2016,16-00015398,Sanitation Code Violation,Other,6959 S PAXTON AVE,60649.0,1.192081e+06,1.859250e+06,5,3,43,41.768766,-87.571478,"(41.768766169704875, -87.57147751207171)",1.0,0.0,312.0,33.0


In [502]:
df4_small_1 = df1[['Population of Multiple Races', 'Household Income by Gross Rent ($000)', 'Owner Occupied Houses']].groupby(df4['Block']).mean().reset_index()

In [598]:
df4_small_2 = df4['Location'].groupby(df4['Block']).count().reset_index()

In [599]:
df4_result = pd.merge(df4_small_1, df4_small_2, on='Block', how="outer")

In [622]:
df4_result = df4_result.rename(columns = {'Location':'Count of sanitation requests'})
df4_result

,Block,Population of Multiple Races,Household Income by Gross Rent ($000),Owner Occupied Houses,Count of sanitationrequests
0,1.0,17.600000,246.285714,179.057143,35
1,2.0,10.764706,264.294118,130.794118,34
2,3.0,15.076923,259.615385,190.000000,13
3,4.0,20.071429,234.642857,183.642857,14
4,5.0,0.000000,150.500000,149.500000,2
5,6.0,34.000000,475.000000,104.000000,1
6,7.0,16.000000,75.000000,186.000000,1


This table shows that most sanitation requests are received from Block 1 and Block 2, followed by a handful of requests from Block 3 and Block 4. This confirms our intial analysis that Block 1 and Block 2 tend to be the most vulnerable blocks. Most of the characteristics of Block 1 and 2 in the Sanitation data are the same as the one in the Vacant Buildings data, except that the mixed race of population rate is lower in this dataset. This means that most sanitation complaints are made by single races such as whites or entirely black. There could also be more environmental hazards in this area, which have caused the residents to complain again and again. One fascinated piece of information is Block number 7. Despite having a low average household income, the number of sanitation requests are very low from this Block. It's possible that houses in this block have better safety measures. Also most people in this Block tend to be the occupied home owners meaning that they take extra care to ensure safety of their houses.

In [612]:
df4.groupby(['Block', 'Creation Date']).count()['Status'].reset_index()

,Block,Creation Date,Status
0,1.0,01/01/2016,3
1,1.0,01/02/2016,6
2,1.0,01/03/2016,9
3,1.0,01/04/2016,17
4,2.0,01/01/2016,3
5,2.0,01/02/2016,6
6,2.0,01/03/2016,17
7,2.0,01/04/2016,8
8,3.0,01/01/2016,1
9,3.0,01/02/2016,1


This analysis on a limited trend shows that Block 1 and Block 2 tend to consistenly receive higher requests, whereas Block 5 and Block 6 and Block 7 tend to receive consistently lower requests.

In [619]:
df1_result

,Block,Population of Multiple Races,Household Income by Gross Rent ($000),Owner Occupied Houses,Count of vacant requests
0,1.0,15.176471,245.441176,136.794118,34
1,2.0,15.542857,242.371429,175.028571,35
2,3.0,18.470588,217.294118,192.176471,17
3,4.0,5.888889,353.888889,162.333333,9
4,5.0,22.333333,293.000000,152.333333,3
5,6.0,0.000000,227.000000,100.000000,1
6,7.0,15.000000,304.000000,282.000000,1


In [620]:
df4_result

,Block,Population of Multiple Races,Household Income by Gross Rent ($000),Owner Occupied Houses,Count of sanitationrequests
0,1.0,17.600000,246.285714,179.057143,35
1,2.0,10.764706,264.294118,130.794118,34
2,3.0,15.076923,259.615385,190.000000,13
3,4.0,20.071429,234.642857,183.642857,14
4,5.0,0.000000,150.500000,149.500000,2
5,6.0,34.000000,475.000000,104.000000,1
6,7.0,16.000000,75.000000,186.000000,1


There are a couple of differences between the blocks that get "vacant" complaints vs. the blocks that get the "sanitation" complaints.

1) Block 1 in the "vacant" data seems to be relatively "richer" than Block 1 in the "sanitation" data
2) Block 4 in the "vacant" data is significantly more prosperous that Block 4 in the "sanitation" data. However, Block 4 in the "vacant" data still generate some complaints. This merits further investigation into other social characteristics such as education.
3) Block 5 and Block 6 both are more prosperous than Block 5 and Block 6 in the "sanitation" data, however the number of requests generated by both blocks for both types is the same. This means that, after controlling for income, we can observe that no matter who lives in these blocks (black, whites, rich, modest etc.), the quality of lives in these blocks is very high.